In [ ]:
# Create subset of the data for quicker testing

In [12]:
import os
import json
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import rioxarray as rxr
from rasterio.crs import CRS
from sqlalchemy import create_engine
import rasterstats

In [13]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
# Original dataset table names
admin_list = [
    'afg_admbnda_adm2',
    'afg_admbnda_adm1',
    'afg_admbnda_region'
]

flood_list = [
    'afg_fldzonea_100k_ncia_v2_029cm_dissolve_all_v2',
    'afg_fldzonea_100k_ncia_v2_121cm_dissolve_all_v2',
    'afg_fldzonea_100k_ncia_v2_271cm_dissolve_all_v2'
]

avalanche_list = [
    'afg_avsa_clean_dissolve'
]

# Clip extent to clip by 
clip_extent = gpd.read_file(r'D:\iMMAP\proj\ASDC\data\Flood\clip_extent.shp')

In [16]:
# Looping over datasets, clips and saves new datasets with _testClip added to table name
def clip(adm, flood):
    flood = adm.clip(flood)
    return flood

def AOI_clipper(dataset_list, clip_extent, db_con, geometry_column_label):
    for i in range(len(dataset_list)):
        dataset = gpd.GeoDataFrame.from_postgis('SELECT * from {}'.format(dataset_list[i]), db_con, geometry_column_label)
        dataset_clipped = clip(dataset, clip_extent)
        dataset_clipped = dataset_clipped.explode() # converting all multipolygons to polygons 
        dataset_clipped.to_postgis('{}_testclip'.format(dataset_list[i]), db_con, if_exists='replace')
        print('{}_testClip - Saved to db'.format(dataset_list[i]))                     

In [17]:
# Run 
#AOI_clipper(admin_list, clip_extent, con, 'geom')
#AOI_clipper(flood_list, clip_extent, con, 'wkb_geometry')
AOI_clipper(avalanche_list, clip_extent, con, 'wkb_geometry')

C:\Users\VMO\AppData\Local\Temp\ipykernel_18384\2283473243.py:10: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  dataset_clipped = dataset_clipped.explode() # converting all multipolygons to polygons


afg_avsa_clean_dissolve_testClip - Saved to db
